# Learn Markdown

In [1]:
!lsmem

RANGE                                 SIZE  STATE REMOVABLE BLOCK
0x0000000040000000-0x00000005bfffffff  22G online       yes 8-183

Memory block size:       128M
Total online memory:      22G
Total offline memory:      0B


In [5]:
import torch as t
import tensorflow as tf

## Quick Start

### First Pass

In [ ]:
# Load a dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
# Build a machine learning model
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

### Second Pass

In [6]:
import tensorflow as tf
print(tf.__version__)

2.14.0


#### Build A model in Torch

In [4]:
import torch as T

In [17]:
class NeuralNetwork(T.nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = T.nn.Flatten()
        self.linear_relu_stack = T.nn.Sequential(
            T.nn.Linear(28*28, 512),
            T.nn.ReLU(),
            T.nn.Linear(512, 512),
            T.nn.ReLU(),
            T.nn.Linear(512, 10),
        )
    def forward(self, x): # Operation on input by forward method
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to("cpu")
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [22]:
X = T.rand(1,28,28,)
logits = model(X)
pred_probab = T.nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([8])


##### Model Layers

In [17]:
input_image = T.rand(3,28,28)
print(input_image.size())

## nn.Flatten
flatten = T.nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

## nn.Linear
from torch import nn
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 28, 28])
torch.Size([3, 784])
torch.Size([3, 20])


In [18]:
## nn.Sequential
seq_modules = T.nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)

#### Build A model in TF

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = tf.keras.layers.Dense(10, activation='relu')
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)
model = MyModel()

## Work With Tensor

**A reminder for `PyTorch`**

In [28]:
import torch as t
torch_randn = t.randn(1,5, dtype=t.double)
print(torch_randn)

tensor([[-0.1673,  1.6550,  0.0093,  1.9103, -1.0395]], dtype=torch.float64)


**Here, with `Tensorflow`**

In [33]:
import tensorflow as tf
import numpy as np

In [50]:
print(tf.math.add(1,1))
print(tf.math.add([1, 2], [3, 4]))
print(tf.math.add( np.array([1]), np.array([2]) ))
print(tf.math.add(np.array([[1,2,3],[1,2,3]]),np.array([[1,2,3],[1,2,3]])))

tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor([3], shape=(1,), dtype=int64)
tf.Tensor(
[[2 4 6]
 [2 4 6]], shape=(2, 3), dtype=int64)


### Work With `DataSets`
> `tf.data.Dataset` API for building data pipeline feeding into model

In [72]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# Create a CSV file
import tempfile
_, filename = tempfile.mkstemp() # filename -> str

with open(filename, 'w') as f: # 
    f.write("""Line 1
Line 2
Line 3
  """)

with open(filename, 'r') as reader:
  print(reader.read())

ds_file = tf.data.TextLineDataset(filename)

Line 1
Line 2
Line 3
  


In [73]:
ds_tensors = ds_tensors.map(tf.math.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

print('Elements of ds_tensors:')
for x in ds_tensors:
  print(x)

print('\nElements in ds_file:')
for x in ds_file:
  print(x)

Elements of ds_tensors:
tf.Tensor([4 9], shape=(2,), dtype=int32)
tf.Tensor([16 25], shape=(2,), dtype=int32)
tf.Tensor([36  1], shape=(2,), dtype=int32)

Elements in ds_file:
tf.Tensor([b'Line 1' b'Line 2'], shape=(2,), dtype=string)
tf.Tensor([b'Line 3' b'  '], shape=(2,), dtype=string)


### Work With `Layers`

by wrapping your code in `tf.function`, which replaces Python iteration with the equivalent graph operations using AutoGraph.
```python
@tf.function
def train(model, dataset, optimizer):
  for x, y in dataset:
    with tf.GradientTape() as tape:
      # training=True is only needed if there are layers with different
      # behavior during training versus inference (e.g. Dropout).
      prediction = model(x, training=True)
      loss = loss_fn(prediction, y)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
```
***BUT*** If you use the Keras Model.fit API, you `won't` have to worry about dataset iteration.

```python
model.compile(optimizer=optimizer, loss=loss_fn)
model.fit(dataset)
```
```python
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.02),
                           input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10)
])

# Model is the full model w/o custom layers
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_data, epochs=NUM_EPOCHS)
loss, acc = model.evaluate(test_data)

print("Loss {}, Accuracy {}".format(loss, acc))
```

`tf.function` provides a way to convert **data-dependent** control flow into graph-mode equivalents like `tf.cond` and `tf.while_loop`.
```python
class DynamicRNN(tf.keras.Model):
    def __init__(self):
        super(DynamicRNN, self).__init__(self)
        self.cell = rnn_cell
    
    @tf.function(input_signature=[tf.TensorSpec( dtype=tf.float32, shape=[None, None, 3] )])
    def call(self,input_data):
        # [batch, time, features] -> [time, batch, features]
        input_data = tf.transpose(input_data, [1, 0, 2])
        timesteps =  tf.shape(input_data)[0]
        batch_size = tf.shape(input_data)[1]
        outputs = tf.TensorArray(tf.float32, timesteps)
        state = self.cell.get_initial_state(batch_size = batch_size, dtype=tf.float32)
        for i in tf.range(timesteps):
            output, state = self.cell(input_data[i], state)
            outputs = outputs.write(i, output)
        return tf.transpose(outputs.stack(), [1, 0, 2]), state
```

## Checkpoints

In [8]:
import tensorflow as tf
class Net(tf.keras.Model):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = tf.keras.layers.Dense(5)
    def call(self, x):
        return self.l1(X)
net = Net()
net.save_weights('ckpts/easy_checkpoint')